In [1]:
!pip install transformers

You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading: 100%|██████████| 758/758 [00:00<00:00, 191kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 607kB/s]
Downloading: 100%|██████████| 252/252 [00:00<00:00, 102kB/s]
Downloading: 100%|██████████| 438M/438M [00:26<00:00, 16.6MB/s]


In [19]:
import pandas as pd

headlines_df = pd.read_csv('../data/analyst_ratings_processed.csv')
headlines_df = headlines_df.sample(300, random_state=42)

In [10]:
headlines_df.head()

,Unnamed: 0,title,date,stock
951347,963395.0,Top Stocks In The Surety & Title Insurance Ind...,2010-04-13 05:00:00-04:00,ORI
98223,102011.0,Asterias Provides 6 Mo. Data Readout From Its ...,2018-07-17 07:05:00-04:00,AST
940133,952152.0,ONEOK Partners to Participate in Bakken Day,2010-08-04 16:22:00-04:00,OKS
185334,189463.0,Shares of Broadsoft to Resume Trade at 4:35PM EST,2013-11-04 16:12:00-05:00,BSFT
524898,535387.0,"Courier Corp Holder Gamco Reports 6.97%, Up Fr...",2015-03-30 16:20:00-04:00,GBL


In [12]:
import numpy as np

headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

print(headlines_list[:5])

['Option Alert: Clean Energy Fuels Shares Spike Higher; Activity in Aug 10, Sep 10 Calls; Currently $9.92', 'Market Update: U.S. Equity Markets are Mixed Going into the Close', "27 Stocks Moving In Tuesday's Mid-Day Session", 'Sotherly Hotels Reports Successful Execution of $35M Secured Loan on Hilton Wilmington Riverside Hotel in Wilmington, North Carolina', 'Stifel Nicolaus Upgrades Navios Maritime Holdings Inc. to Buy, Announces $14.00 PT']


In [15]:
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')

In [17]:
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([300, 3])


In [18]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.9402, 0.0254, 0.0344],
        [0.2727, 0.6773, 0.0500],
        [0.0830, 0.7072, 0.2098],
        [0.9374, 0.0175, 0.0451],
        [0.4195, 0.0178, 0.5627],
        [0.8486, 0.0541, 0.0973],
        [0.0215, 0.2806, 0.6979],
        [0.1797, 0.0092, 0.8111],
        [0.8760, 0.0111, 0.1129],
        [0.7264, 0.0188, 0.2548],
        [0.0858, 0.0994, 0.8148],
        [0.0789, 0.0100, 0.9112],
        [0.7675, 0.0193, 0.2131],
        [0.0080, 0.9702, 0.0218],
        [0.9395, 0.0110, 0.0495],
        [0.8589, 0.1016, 0.0395],
        [0.7818, 0.1004, 0.1177],
        [0.0080, 0.9754, 0.0166],
        [0.0580, 0.2505, 0.6915],
        [0.2982, 0.4383, 0.2635],
        [0.0094, 0.9708, 0.0198],
        [0.9154, 0.0385, 0.0461],
        [0.0365, 0.0528, 0.9107],
        [0.0282, 0.6327, 0.3391],
        [0.0243, 0.1238, 0.8520],
        [0.0350, 0.0672, 0.8978],
        [0.0246, 0.6180, 0.3574],
        [0.0368, 0.1453, 0.8179],
        [0.0238, 0.1664, 0.8099],
        [0.009

In [20]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

In [41]:
df.sort_values(by='Positive', ascending=False)

,Headline,Positive,Negative,Neutral
249,"Earnings Preview: Stryker EPS, Revenue Expected to Rise from Previous Quarter",0.955498,0.020781,0.023721
172,"Asterias Provides 6 Mo. Data Readout From Its AST-OPC1 Phase 1/2a Clinical Trial, Says Results Continue to Demonstrate Favorable Safety Profile, Potential for Durable Cell Engraftment at the Injury Site and Improved Motor Function",0.954589,0.014721,0.030690
133,Raymond James Says Client AUM Grew 4% Y/Y To $505.9B In November,0.951677,0.016788,0.031535
164,"Courier Corp Holder Gamco Reports 6.97%, Up From 0.05% Pn 12/31/2014",0.951596,0.027925,0.020479
162,"SAExploration shares are trading higher after the company late Monday reported a combined $130 million of new projects om Alaska, Canada and the Middle East.",0.950563,0.018216,0.031221
132,"Astro-Med Q2 EPS $0.17 Up From $0.11 YoY, Sales $33.8M Up From $27.483M YoY",0.950424,0.026136,0.023440
263,Radware Awarded Contract by Leading US National Cellular Network Provider,0.950311,0.014512,0.035177
125,U.S. Government Expands BioThrax Procurement Contract to 17.92M Doses from 14.5M Increasing Contract Value by up to $101 Million (EBS),0.947236,0.023031,0.029733
171,"Bank of America Upgrades GWR, And WERN From Underperform To Neutral",0.946677,0.030561,0.022762
288,Sino-Global Signs Two Year General Agency Service Agreement; Expected To Add Approximately $12M In Revenue Annually,0.944380,0.016400,0.039221
